In [ ]:
# ✅ Signature Verification with CEDAR Dataset using Sigmoid Siamese Network
# Google Colab-ready full code — Fresh Build with Your Requirements

import os
import zipfile
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, Input, Model, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from google.colab import files
from IPython.display import clear_output

# --------------------------------------
# 📁 STEP 1: Upload CEDAR ZIP File
# --------------------------------------
print("\n📁 Upload CEDAR ZIP file (genuine signatures):")
cedar_zip = files.upload()
cedar_zip_path = list(cedar_zip.keys())[0]

os.makedirs("data/cedar", exist_ok=True)
with zipfile.ZipFile(cedar_zip_path, 'r') as zip_ref:
    zip_ref.extractall("data/cedar")

# Load and index all extracted files
all_cedar_files = sorted([f for f in os.listdir("data/cedar") if f.endswith(".png") or f.endswith(".jpg")])



📁 Upload CEDAR ZIP file (genuine signatures):


Saving cedar.zip to cedar.zip


In [ ]:
import random
user_signatures = {}
used_files = set()

while True:
    user_name = input("\n👤 Enter user name: ").strip()
    print(f"🔁 Please upload 5 signature images for {user_name} using drag & drop:")
    uploaded_files = files.upload()

    sigs = []
    for fname in uploaded_files:
        if fname not in used_files:
            img = cv2.imread(os.path.join("data/cedar", fname), cv2.IMREAD_GRAYSCALE)
            if img is None:
                img = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (220, 155)) / 255.0
            sigs.append(img)
            used_files.add(fname)
            if len(sigs) == 5:
                break

    if len(sigs) < 5:
        print(f"❌ Only {len(sigs)} valid signatures uploaded for {user_name}. Skipping user.")
    else:
        user_signatures[user_name] = sigs
        print(f"✅ {user_name} added with 5 signatures.")

    more = input("Add more users? (y/n): ").strip().lower()
    if more != 'y':
        break
    clear_output()

# --------------------------------------
# 🧪 STEP 3: Create Training Pairs
# --------------------------------------
def make_pairs(user_signatures):
    X1, X2, y = [], [], []
    users = list(user_signatures.keys())

    for user in users:
        real_imgs = user_signatures[user]
        for i in range(len(real_imgs)):
            for j in range(i+1, len(real_imgs)):
                X1.append(real_imgs[i])
                X2.append(real_imgs[j])
                y.append(1)

            # Add random negative pair from different user
            other_user = random.choice([u for u in users if u != user])
            forged_img = random.choice(user_signatures[other_user])
            X1.append(real_imgs[i])
            X2.append(forged_img)
            y.append(0)

    X1 = np.array(X1).reshape(-1, 155, 220, 1)
    X2 = np.array(X2).reshape(-1, 155, 220, 1)
    y = np.array(y, dtype=np.float32)
    return X1, X2, y

X1, X2, y = make_pairs(user_signatures)
X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(X1, X2, y, test_size=0.2, random_state=42)

# --------------------------------------
# 🧠 STEP 4: Build Sigmoid Siamese Network
# --------------------------------------
def build_siamese():
    def feature_extractor():
        model = models.Sequential()
        model.add(layers.Conv2D(64, (3,3), activation='relu', input_shape=(155, 220, 1)))
        model.add(layers.MaxPooling2D(2))
        model.add(layers.Conv2D(128, (3,3), activation='relu'))
        model.add(layers.MaxPooling2D(2))
        model.add(layers.Flatten())
        model.add(layers.Dense(128, activation='relu'))
        return model

    input_a = Input(shape=(155, 220, 1))
    input_b = Input(shape=(155, 220, 1))
    feature = feature_extractor()

    encoded_a = feature(input_a)
    encoded_b = feature(input_b)

    L1_layer = layers.Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_a, encoded_b])
    output = layers.Dense(1, activation='sigmoid')(L1_distance)

    model = Model(inputs=[input_a, input_b], outputs=output)
    return model

model = build_siamese()
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(0.0001), metrics=['accuracy'])
model.summary()

# --------------------------------------
# 🏋️ STEP 5: Train the Model
# --------------------------------------
callbacks = [EarlyStopping(patience=5, restore_best_weights=True), ReduceLROnPlateau()]
model.fit([X1_train, X2_train], y_train, validation_data=([X1_test, X2_test], y_test),
          epochs=30, batch_size=16, callbacks=callbacks)

# --------------------------------------
# 🔍 STEP 6: Signature Verification
# --------------------------------------
def verify_signature(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print("❌ Could not read the image.")
        return
    img = cv2.resize(img, (220, 155)) / 255.0
    img = img.reshape(1, 155, 220, 1)

    scores = {}
    for user, refs in user_signatures.items():
        user_scores = []
        for ref in refs:
            ref = ref.reshape(1, 155, 220, 1)
            score = model.predict([img, ref])[0][0]
            user_scores.append(score)
        scores[user] = np.mean(user_scores)

    best_match = max(scores, key=scores.get)
    if scores[best_match] > 0.5:
        print(f"✅ Match found: {best_match} (Confidence: {scores[best_match]:.2f})")
    else:
        print("❌ Signature not recognized in the database.")

# --------------------------------------
# 📄 STEP 7: Upload Signature to Verify
# --------------------------------------
print("\n📄 Upload a signature image to verify:")
test_file = files.upload()
for fname in test_file:
    verify_signature(fname)



👤 Enter user name: pulkit
🔁 Please upload 5 signature images for pulkit using drag & drop:


Saving 1st.jpg to 1st (1).jpg
Saving 2nd - Copy.jpg to 2nd - Copy (1).jpg
Saving 2nd.jpg to 2nd (1).jpg
Saving 3rd - Copy.jpg to 3rd - Copy (1).jpg
Saving 3rd.jpg to 3rd (1).jpg
✅ pulkit added with 5 signatures.
Add more users? (y/n): n


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 155, 220,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 155, 220,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 128)       │ 32,203,648 │ input_layer[0][0… │
│ (Sequential)        │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 128)       │          0 │ sequential[0][0], │
│                     │                   │            │ sequential[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │        129 │ lambda[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 32,203,777 (122.85 MB)

 Trainable params: 32,203,777 (122.85 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step - accuracy: 0.3264 - loss: 0.6823 - val_accuracy: 0.5000 - val_loss: 0.6235 - learning_rate: 1.0000e-04
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 4s/step - accuracy: 0.3750 - loss: 0.6162 - val_accuracy: 0.5000 - val_loss: 0.5767 - learning_rate: 1.0000e-04
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.5278 - loss: 0.5561 - val_accuracy: 0.6667 - val_loss: 0.5406 - learning_rate: 1.0000e-04
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step - accuracy: 0.7500 - loss: 0.5098 - val_accuracy: 0.6667 - val_loss: 0.4995 - learning_rate: 1.0000e-04
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/step - accuracy: 0.8681 - loss: 0.4453 - val_accuracy: 0.8333 - val_loss: 0.4632 - learning_rate: 1.0000e-04
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.9236 - loss: 0.4171 - val_accuracy: 0.8333 - val_loss: 0.4280 - learning_rate: 1.0000e-04
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.9514 - loss: 0.3953 - val_accu

Saving IMG-20250327-WA0016.jpg to IMG-20250327-WA0016.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
✅ Match found: bilal (Confidence: 0.88)


In [ ]:
print("\n📄 Upload a signature image to verify:")
test_file = files.upload()
for fname in test_file:
    verify_signature(fname)


📄 Upload a signature image to verify:


Saving test4tg.jpg to test4tg.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
❌ Signature not recognized in the database.
